### Part 1. Data Setup and Asset Creation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import norm
from collections import OrderedDict
import sys, os
from prisk.asset import PowerPlant, WareHouse, Manufacturing

##### Step 1. Load assets with flood depth information, asset value and ownership, as well as flood protection levels

In [18]:
# Load CSVs
assets = pd.read_csv(r"C:\Users\Mark.DESKTOP-UFHIN6T\OneDrive - Nexus365\CFRF\2025\Deals_WG\PRISK\synthetic\synthetic_assets_thailand_flooded.csv")
# Make RP columns int not str1
assets.columns = assets.columns.map(lambda c: int(c) if str(c).isdigit() else c)
# Load first five rows
assets.head()

,asset_id,asset_name,asset_class,size,size_unit,asset_value_usd,owner_id,owner_name,country,latitude,...,2,5,10,25,50,100,200,500,1000,flood_protection
0,THA-00001,Power Asset 00001,power,924.51,MW,5.028195e+07,THO-059,Thai Holdings 059,Thailand,17.327446,...,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,16.861799
1,THA-00002,Power Asset 00002,power,109.35,MW,6.232471e+07,THO-024,Thai Holdings 024,Thailand,18.493527,...,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,18.905199
2,THA-00003,Power Asset 00003,power,1284.85,MW,6.203643e+07,THO-029,Thai Holdings 029,Thailand,19.246097,...,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,17.822800
3,THA-00004,Power Asset 00004,power,1472.55,MW,1.009066e+08,THO-050,Thai Holdings 050,Thailand,7.587059,...,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,16.318800
4,THA-00005,Power Asset 00005,power,111.74,MW,1.044381e+08,THO-048,Thai Holdings 048,Thailand,17.863367,...,11.1,18.24,19.45,20.52,21.28,22.0,22.69,23.47,23.93,17.960400


##### Step 2. Load financial data and ownership structure

In [19]:
financial_data = pd.read_csv(r"C:\Users\Mark.DESKTOP-UFHIN6T\OneDrive - Nexus365\CFRF\2025\Deals_WG\PRISK\synthetic\synthetic_owners_thailand.csv")
financial_data.head()

,owner_id,owner_name,leverage_ratio,source,url
0,THO-001,Thai Holdings 001,0.067,synthetic,NaN
1,THO-002,Thai Holdings 002,0.404,synthetic,NaN
2,THO-003,Thai Holdings 003,0.319,synthetic,NaN
3,THO-004,Thai Holdings 004,0.231,synthetic,NaN
4,THO-005,Thai Holdings 005,0.050,synthetic,NaN


##### Step 3. Extract leverage ratios for each firm

In [20]:
leverage_ratios = {firm: leverage for firm, leverage in zip(financial_data["owner_name"], financial_data["leverage_ratio"])}
# Print first one
print(next(iter(leverage_ratios)), next(iter(leverage_ratios.values())))

Thai Holdings 001 0.067


##### Step 4: Load damage curves

In [21]:
damage_curves = pd.read_excel("https://kuleuven-prisk.s3.eu-central-1.amazonaws.com/damage_curves.xlsx")
damage_curves.head()

,depth,huizinga_base,huizinga_steep,huizinga_flat,huizinga_shallow,huizinga_very_steep,damage,production,linear,low,very_low,medium_high,highest,medium
0,0.00,0.000,0.000,0.00,0.00,0.00,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.05,0.028,0.035,0.02,0.01,0.04,0.004,0.5,0.005,0.000083,0.000001,0.075959,0.190152,0.038553
2,0.10,0.056,0.070,0.04,0.02,0.08,0.008,1.0,0.010,0.000330,0.000010,0.095703,0.205375,0.054522
3,0.15,0.084,0.105,0.06,0.03,0.12,0.012,1.5,0.015,0.000742,0.000035,0.109552,0.214839,0.066776
4,0.20,0.112,0.140,0.08,0.04,0.16,0.016,2.0,0.020,0.001320,0.000084,0.120578,0.221817,0.077106


In [24]:
def extract_firms(assets, asset_type, damage_curves, leverage_ratios={}, discount_rate=0.02, time_horizon=25, production_period):
    '''
    For a given asset-type, it loads the asset-specific damage curves, asset-level information, and maps them to firms.

    Parameters
    ----------
    assets : df (asset-level data dataframe)
    asset_type: string (type of asset e.g. PowerPlant)
    damage_curves: df (asset-specific damage curves)
    leverage_ratios: dic (company leverage ratios)
    discount_rate: float (discount rate with which to calculate NPV)
    time_horizone: int (time horizon for the analysis (and for NPV calc))
    production_period: int (over what period are we calculating production (0=year, 365=day, 8760=hour))
    '''
    assets.sort_values("owner_name", inplace=True)
    # Filter assets by asset type
    try:
        asset_subset = assets[assets['asset_class']==asset_type].copy()

SyntaxError: non-default argument follows default argument (2702188774.py, line 1)

In [22]:
def extract_firms(assets, damage_curves=None, leverage_ratios={}, discount_rate=0.05, time_horizon=25):
    assets.sort_values("owner_name", inplace=True)
    if damage_curves is None:
        damage_curves = continuous_curves
    # Create assets for each asset type
    # Starting with power assets
    power_assets = assets[assets['asset_class']=='power'].copy()
    # Power specific variables
    power_unit_price = 60 # pounds per MWh
    power_margin = 0.2 # 20% profit margin for electricity generation
    power_assets.loc[:, "asset"] = power_assets.apply(lambda x: 
                                                PowerPlant(
                                                    name=x["asset_name"],
                                                    flood_damage_curve=damage_curves,
                                                    flood_exposure=[FloodExposure(return_period, x[return_period]) 
                                                                    for return_period in return_period_columns if x[return_period] > 0],
                                                    flood_protection = x["flood_protection"],
                                                    production_path=np.repeat(x["size"]*24*365, time_horizon),
                                                    replacement_cost=x["asset_value_usd"],
                                                    unit_price=power_unit_price,
                                                    discount_rate=discount_rate,
                                                    margin=power_margin,
                                                  ), axis=1)
    # Warehouse
    warehouse_assets = assets[assets['asset_class']=='warehouse'].copy()
    # Warehouse specific variables
    warehouse_unit_price = 100 # pounds p.a. rental income
    warehouse_margin = 0.09 # 9% profit margin of warehouse facilities
    warehouse_assets.loc[:, "asset"] = warehouse_assets.apply(lambda x: 
                                                    WareHouse(
                                                        name=x["asset_name"],
                                                        flood_damage_curve=damage_curves,
                                                        flood_exposure=[FloodExposure(return_period, x[return_period]) 
                                                                        for return_period in return_period_columns if x[return_period] > 0],
                                                        flood_protection = x["flood_protection"],
                                                        production_path=np.repeat(x["size"], time_horizon),
                                                        replacement_cost=x["asset_value_usd"],
                                                        unit_price=warehouse_unit_price,
                                                        discount_rate=discount_rate,
                                                        margin=warehouse_margin,
                                                      ), axis=1)
    # Manufacturing facility
    manufacturing_assets = assets[assets['asset_class']=='manufacturing'].copy()
    # Manufacturing specific variables
    manufacturing_unit_price = 2500 # pounds p.a. production per m^2
    manufacturing_margin = 0.07 # 7% profit margin
    manufacturing_assets.loc[:, "asset"] = manufacturing_assets.apply(lambda x: 
                                                    Manufacturing(
                                                    name=x["asset_name"],
                                                    flood_damage_curve=damage_curves,
                                                    flood_exposure=[FloodExposure(return_period, x[return_period]) 
                                                                    for return_period in return_period_columns if x[return_period] > 0],
                                                    flood_protection = x["flood_protection"],
                                                    production_path=np.repeat(x["size"], time_horizon),
                                                    replacement_cost=x["asset_value_usd"],
                                                    unit_price=manufacturing_unit_price,
                                                    discount_rate=discount_rate,
                                                    margin=manufacturing_margin,
                                                  ), axis=1)
    # Combine these back together
    assets = pd.concat(
        [power_assets, warehouse_assets, manufacturing_assets],
        axis=0,
        ignore_index=True
    )
    
    list_of_owners = []
    for owners in assets["owner_name"].unique():
        if pd.isna(owners):
            continue
        list_of_owners.append(owners)
    list_of_owners = list(OrderedDict.fromkeys(list_of_owners))
    owner_map = {owner: Holding(owner, leverage_ratio=leverage_ratios.get(owner)) for owner in list_of_owners}
    holdings = []
    for i, owner in enumerate(assets["owner_name"]):
        if pd.isna(owner):
            continue
        share = 100 # here assuming each asset only has 1 owner. In original PRISK there could be a multi-ownership structure
        holding = owner_map[owner]
        holding.add_asset(assets.loc[i, "asset"], share)
        holdings.append(holding)
    return list(OrderedDict.fromkeys(holdings)), assets

In [23]:
firms, processed_assets = extract_firms(assets, leverage_ratios=leverage_ratios)

NameError: name 'continuous_curves' is not defined